In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 27 23:10:51 2020

@author: GhilmanFatih
"""

import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import pickle

pd.set_option('display.max_columns', None)  
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
url = 'http://www.dpr.go.id/'
with open('dpr.pickle', 'rb') as f:
    ll, m = pickle.load(f)

In [ ]:
url_anggota = url+'anggota'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
response = requests.get(url_anggota, headers=agent)
soup = BeautifulSoup(response.content,"html.parser")

In [ ]:
table = soup.find( "table", {"id":"data-anggota2"} )
table_rows = table.find_all('tr')

ll = []
for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.renderContents().decode('UTF-8') for tr in td]
    row.append(url + str(tr).split("'")[1])
    ll.append(row)

In [3]:
raw = pd.DataFrame(ll, columns=["noAnggota", 'link_foto', "nama_fraksi_dapil", "akd", 'link_profil'])

In [4]:
raw.tail()

,noAnggota,link_foto,nama_fraksi_dapil,akd,link_profil
567,571,"<img class=""img-responsive"" src=""/doksigota/photo/1587.jpg""/>","Hj. ALIYAH MUSTIKA ILHAM, SE<br/>Fraksi Partai Demokrat<br/>SULAWESI SELATAN I",Komisi IX,http://www.dpr.go.id//blog/profil/id/1587
568,572,"<img class=""img-responsive"" src=""/doksigota/photo/1830.jpg""/>",MUHAMMAD DHEVY BIJAK<br/>Fraksi Partai Demokrat<br/>SULAWESI SELATAN III,Komisi IX,http://www.dpr.go.id//blog/profil/id/1830
569,573,"<img class=""img-responsive"" src=""/doksigota/photo/1831.jpg""/>",RUSDA MAHMUD<br/>Fraksi Partai Demokrat<br/>SULAWESI TENGGARA,Komisi VII,http://www.dpr.go.id//blog/profil/id/1831
570,574,"<img class=""img-responsive"" src=""/doksigota/photo/1832.jpg""/>","DR. H. SUHARDI DUKA, M.M.<br/>Fraksi Partai Demokrat<br/>SULAWESI BARAT",Komisi IV<br/>Badan Anggaran,http://www.dpr.go.id//blog/profil/id/1832
571,575,"<img class=""img-responsive"" src=""/doksigota/photo/1618.jpg""/>","WILLEM WANDIK, S.Sos<br/>Fraksi Partai Demokrat<br/>PAPUA",Komisi V<br/>Badan Kerja Sama Antar Parlemen,http://www.dpr.go.id//blog/profil/id/1618


In [5]:
# preprocess link untuk foto
raw['link_foto'] = url + raw['link_foto'].str.replace(r'(.+)src="(.+)".+', r'\2')

In [6]:
raw["nama"] = raw["nama_fraksi_dapil"].str.split("<br/>", n = 3, expand = True)[0]
raw["fraksi"] = raw["nama_fraksi_dapil"].str.split("<br/>", n = 3, expand = True)[1]
raw["dapil"] = raw["nama_fraksi_dapil"].str.split("<br/>", n = 3, expand = True)[2]

In [7]:
raw['akd_clean'] = [list(item) for item in [set(x) for x in raw['akd'].str.split('<br/>')]]
#raw['akd_clean'] = [','.join(map(str, list(item))) for item in [set(x) for x in raw['akd'].str.split('<br/>')]]

In [8]:
# drop unnecessary columns
raw.drop('akd', axis=1, inplace=True)
raw.drop('nama_fraksi_dapil', axis=1, inplace=True)

In [ ]:
m = []
for i in range(len(raw)):
    l = []
    print(i)
    l.append(raw.noAnggota[i])
    l.append(raw.nama[i])
    try:
        response = requests.get(raw.link_profil[i], headers=agent)
        soup = BeautifulSoup(response.content,"html.parser")
        table = soup.find( "div", {"class":"col-md-9 col-md-pull-4 mb30"} )
        table_rows = table.find_all('div')
        for tr in table_rows[1:]:
            td = tr.find_all('div')
            row = [tr.renderContents().decode('UTF-8') for tr in td]
            if len(row)>0:
                l.append(row)
        m.append(list( l[i] for i in [0,1,4,5,6,8,10] ))
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")


In [ ]:
# with open('dpr.pickle', 'wb') as f:
#     pickle.dump([ll, m], f)

In [9]:
detail_profil = pd.DataFrame(m, columns=['noAnggota', 'nama', 'ttl', 'agama', 'pendidikan', 'pekerjaan', 'organisasi'])

In [10]:
detail_profil['ttl'] = [x[1].strip(': ') for x in detail_profil.ttl]

In [11]:
# delete data yang ttl-nya salah
detail_profil = detail_profil[-detail_profil.ttl.str.contains('2019')]

In [12]:
detail_profil['kotaLahir'] = detail_profil['ttl'].str.replace(r'(.+) \/ .+([\d]{4})', r'\1')
detail_profil['usia'] = datetime.now().year - pd.to_numeric(detail_profil['ttl'].str.replace(r'(.+) \/ .+([\d]{4})', r'\2'))

In [13]:
detail_profil['agama'] = [x[1].strip(': ') for x in detail_profil.agama]

In [14]:
detail_profil['pendidikan'] = [x[0] for x in detail_profil.pendidikan]

In [15]:
detail_profil['pekerjaan'] = [x[0] for x in detail_profil.pekerjaan]
detail_profil['organisasi'] = [x[0] for x in detail_profil.organisasi]

In [16]:
profil_raw = raw.merge(detail_profil, how='left', on=['noAnggota', 'nama'])

In [17]:
profil_raw['pendidikan_terakhir'] = profil_raw.pendidikan.str.replace(r'(.+)<li>(.+)<\/li>.+', r'\2')
profil_raw['pendidikan_terakhir'] = profil_raw.pendidikan_terakhir.str.replace('\n<div class="stitle">Riwayat Pendidikan</div>\n<ul class="custom-button">\n', '').str.strip('<li>\n')
profil_raw['pendidikan_terakhir'] = profil_raw.pendidikan_terakhir.str.split(',', expand=True)[0]

In [18]:
profil_raw['pendidikan_terakhir'] = np.where(profil_raw.pendidikan_terakhir.isnull(),'0_no_data',
    np.where(profil_raw.pendidikan_terakhir.str.contains('smp', case=False), '1_smp', 
    np.where(profil_raw.pendidikan_terakhir.str.contains('sma', case=False), '2_sma',
    np.where(profil_raw.pendidikan_terakhir.str.contains('diploma', case=False), '3_d3',
    np.where(profil_raw.pendidikan_terakhir.str.contains('s1', case=False), '4_s1',
    np.where(profil_raw.pendidikan_terakhir.str.contains('s2', case=False), '5_s2',
    np.where(profil_raw.pendidikan_terakhir.str.contains('s3', case=False), '6_s3', '0_no_data')))))))


In [19]:
profil_raw['is_kader'] = np.where(((profil_raw.fraksi.str.contains('amanat nasional', case=False)) & (profil_raw.organisasi.str.contains('PAN') | profil_raw.organisasi.str.contains('amanat nasional', case=False))) |
    ((profil_raw.fraksi.str.contains('demokrasi indonesia perjuangan', case=False)) & (profil_raw.organisasi.str.contains('PDIP') | profil_raw.organisasi.str.contains('demokrasi indonesia perjuangan', case=False))) |
    ((profil_raw.fraksi.str.contains('demokrat', case=False)) & (profil_raw.organisasi.str.contains('PD') | profil_raw.organisasi.str.contains('demokrat', case=False))) |
    ((profil_raw.fraksi.str.contains('gerakan indonesia raya', case=False)) & (profil_raw.organisasi.str.contains('gerindra', case=False) | profil_raw.organisasi.str.contains('gerakan indonesia raya', case=False))) |
    ((profil_raw.fraksi.str.contains('golongan karya', case=False)) & (profil_raw.organisasi.str.contains('golkar', case=False) | profil_raw.organisasi.str.contains('golongan karya', case=False))) |
    ((profil_raw.fraksi.str.contains('keadilan sejahtera', case=False)) & (profil_raw.organisasi.str.contains('PKS') | profil_raw.organisasi.str.contains('keadilan sejahtera', case=False))) |
    ((profil_raw.fraksi.str.contains('kebangkitan bangsa', case=False)) & (profil_raw.organisasi.str.contains('PKB') | profil_raw.organisasi.str.contains('kebangkitan bangsa', case=False))) |
    ((profil_raw.fraksi.str.contains('nasdem', case=False)) & (profil_raw.organisasi.str.contains('nasdem', case=False) | profil_raw.organisasi.str.contains('nasional demokrat', case=False))) |
    ((profil_raw.fraksi.str.contains('persatuan pembangunan', case=False)) & (profil_raw.organisasi.str.contains('PPP') | profil_raw.organisasi.str.contains('Persatuan Pembangunan', case=False)))
    , 'internal', np.where(profil_raw.organisasi.isnull(), 'no_data', 'eksternal'))

In [20]:
profil_raw['is_dewan'] = np.where(
    (profil_raw.pekerjaan.str.contains('DPD')) | 
    (profil_raw.pekerjaan.str.contains('DPR')) | 
    (profil_raw.pekerjaan.str.contains('MPR')) | 
    (profil_raw.pekerjaan.str.contains('DPRD')), 'dewan', 
    np.where(profil_raw.organisasi.isnull(), 'no_data','non-dewan'))

In [21]:
profil_raw['usia_kategori'] = pd.qcut(profil_raw['usia'], q=5, precision=0)

In [22]:
c = profil_raw.groupby('fraksi')['noAnggota'].nunique().rank(ascending=False).reset_index(name='rank_partai')
profil_raw = profil_raw.merge(c, on='fraksi')

In [23]:
profil_raw['partai'] = np.where(profil_raw.fraksi.str.contains('amanat nasional', case=False), 'PAN',
    np.where(profil_raw.fraksi.str.contains('demokrasi indonesia perjuangan', case=False), 'PDIP', 
    np.where(profil_raw.fraksi.str.contains('demokrat', case=False), 'PD',  
    np.where(profil_raw.fraksi.str.contains('gerakan indonesia raya', case=False), 'Gerindra', 
    np.where(profil_raw.fraksi.str.contains('golongan karya', case=False), 'Golkar', 
    np.where(profil_raw.fraksi.str.contains('keadilan sejahtera', case=False), 'PKS',
    np.where(profil_raw.fraksi.str.contains('kebangkitan bangsa', case=False), 'PKB', 
    np.where(profil_raw.fraksi.str.contains('nasdem', case=False), 'Nasdem',
    np.where(profil_raw.fraksi.str.contains('persatuan pembangunan', case=False), 'PPP', 'no_data')))))))))

In [24]:
# exploding akd_clean so we can analyse akd data from array data into long data
profil_long = profil_raw.explode('akd_clean')

In [25]:
def univar_analysis(col):
    c = profil_raw.groupby([col])[col].size().reset_index(name='cnt')
    c['pct'] = round(100 * c['cnt'] / c.cnt.sum(),2)
    return c.sort_values('pct', ascending=False)

In [26]:
univar_analysis('partai')

,partai,cnt,pct
5,PDIP,127,22.20
1,Golkar,85,14.86
0,Gerindra,76,13.29
2,Nasdem,59,10.31
6,PKB,58,10.14
4,PD,54,9.44
7,PKS,50,8.74
3,PAN,44,7.69
8,PPP,19,3.32


In [27]:
univar_analysis('agama')

,agama,cnt,pct
2,Islam,465,83.63
4,Kristen,52,9.35
3,Katolik,24,4.32
1,Hindu,11,1.98
0,Budha,4,0.72


In [28]:
univar_analysis('pendidikan_terakhir')

,pendidikan_terakhir,cnt,pct
5,5_s2,218,38.11
4,4_s1,193,33.74
6,6_s3,84,14.69
0,0_no_data,37,6.47
2,2_sma,32,5.59
3,3_d3,6,1.05
1,1_smp,2,0.35


In [29]:
univar_analysis('is_dewan')

,is_dewan,cnt,pct
0,dewan,288,50.35
2,non-dewan,268,46.85
1,no_data,16,2.80


In [30]:
univar_analysis('is_kader')

,is_kader,cnt,pct
0,eksternal,322,56.29
1,internal,234,40.91
2,no_data,16,2.80


In [31]:
profil_raw['usia'].agg(['mean', 'max', 'min'])

mean    51.757194
max     81.000000
min     24.000000
Name: usia, dtype: float64

In [32]:
profil_raw[(profil_raw.usia==24) | (profil_raw.usia==81)]

,noAnggota,link_foto,link_profil,nama,fraksi,dapil,akd_clean,ttl,agama,pendidikan,pekerjaan,organisasi,kotaLahir,usia,pendidikan_terakhir,is_kader,is_dewan,usia_kategori,rank_partai,partai
390,394,http://www.dpr.go.id//doksigota/photo/1870.jpg,http://www.dpr.go.id//blog/profil/id/1870,"HILLARY BRIGITTA LASUT, S.H., LLM",Fraksi Partai NasDem,SULAWESI UTARA,[Komisi I],Manado / 22 Mei 1996,Katolik,"\n<div class=""stitle"">Riwayat Pendidikan</div>\n<ul class=""custom-button"">\n<li>SD , KAT 01 ST THERST. Tahun: 2003 - 2009</li><li>SMP , MANADO INTERNASIONAL . Tahun: 2009 - 2011</li><li>SMA , SMU 1 MANADO . Tahun: 2011 - 2014</li><li>S1 ILMU HUKUM, UNIV PELITA HARAPAN. Tahun: 2014 - 2018</li><li>S2 , WASHINGTON SCHOLL OF LOW. Tahun: 2018 - 2019</li> </ul>\n","\n<div class=""stitle"">Riwayat Pekerjaan</div>\n<ul class=""custom-button"">\n<li>CV MUTIARA SAKTI , Sebagai: OPERASIONAL DIREKTUR . Tahun: 2019 - 2019</li><li>LIPPO MALL PURI , Sebagai: ARTA OFFICER . Tahun: 2018 - 2019</li><li>AKHH LOW FI, Sebagai: MANGONG . Tahun: 2017 - 2017</li> </ul>\n","\n<div class=""stitle"">Riwayat Organisasi</div>\n<ul class=""custom-button"">\n<li>OSIS SMAN 1 MANADO , Sebagai: KETUA . Tahun: - </li><li>PASKIBRAKA KOTA MANADO , Sebagai: . Tahun: - </li><li>HIMA FH.UPH , Sebagai: BENDAHARA . Tahun: - </li><li>C.P NASDEM SULUT , Sebagai: BENDAHARA . Tahun: - </li> </ul>\n",Manado,24.0,5_s2,internal,non-dewan,"(23.0, 44.0]",4.0,Nasdem
497,501,http://www.dpr.go.id//doksigota/photo/1784.jpg,http://www.dpr.go.id//blog/profil/id/1784,"Hj. FARAH PUTERI NAHLIA, B.A, M.Sc.",Fraksi Partai Amanat Nasional,JAWA BARAT IX,"[Badan Anggaran, Komisi I]",Semarang / 02 Januari 1996,Islam,"\n<div class=""stitle"">Riwayat Pendidikan</div>\n<ul class=""custom-button"">\n<li>SD , Al Kamal, Kebon Jeruk. Tahun: 2001 - 2007</li><li>SMP , Al Azhar Bumi Serpong Damai, Tanggerang. Tahun: 2007 - 2010</li><li>SMA , Sinarmas World Acedemy. Tahun: 2010 - 2012</li><li>DIPLOMA , University Foundation, David Game College, London. Tahun: 2013 - 2016</li><li>S1 , Royal holloway, University Of London. Tahun: 2013 - 2016</li><li>S2 , Royal holloway, University Of London. Tahun: 2016 - 2017</li> </ul>\n","\n<div class=""stitle"">Riwayat Pekerjaan</div>\n<ul class=""custom-button"">\n<li>Masa Studio, Sebagai: Komisaris Utama. Tahun: 2018 - 2018</li><li>Kementerian Luar Negri, Sebagai: Kementerian Luar Negeri, Direktorat HAM dan Kemanusiaan (Intership) Direktorat Jendral Kerjasama Multilateral. Tahun: 2018 - 2018</li> </ul>\n","\n<div class=""stitle"">Riwayat Organisasi</div>\n<ul class=""custom-button"">\n<li>Habitat For Humanity Indonesia, Sebagai: Anggota. Tahun: 2011 - 2012</li><li>English Teaching Programme, Sebagai: Relawan. Tahun: 2011 - 2011</li><li>OSIS SMP Al Azhar BSD, Sebagai: Wakil Bendahara. Tahun: 2008 - 2009</li> </ul>\n",Semarang,24.0,5_s2,eksternal,non-dewan,"(23.0, 44.0]",8.0,PAN
520,524,http://www.dpr.go.id//doksigota/photo/499.jpeg,http://www.dpr.go.id//blog/profil/id/499,"H. ABDUL WAHAB DALIMUNTHE, SH",Fraksi Partai Demokrat,SUMATERA UTARA I,[Komisi VII],Rantau Rapat / 10 Januari 1939,Islam,"\n<div class=""stitle"">Riwayat Pendidikan</div>\n<ul class=""custom-button"">\n<li>SD , SD Rantau Prapat . Tahun: 1948 - 1954</li><li>SMP , SMP Rantau Prapat. Tahun: 1954 - 1957</li><li>SMA , SMA Negeri III Medan . Tahun: 1957 - 1960</li><li>S1 Hukum, Universitas Sumatera Utara . Tahun: 1960 - 1965</li> </ul>\n","\n<div class=""stitle"">Riwayat Pekerjaan</div>\n<ul class=""custom-button"">\n<li>DPR/MPR RI , Sebagai: Anggota FPD DPR RI . Tahun: 2014 - 2019</li><li>DPR/MPR RI , Sebagai: Anggota FPD DPR RI . Tahun: 2009 - 2014</li><li>DPRD Provinsi Sumatera Utara , Sebagai: Ketua DPRD . Tahun: 2004 - 2009</li><li>Pemerintah Provinsi Sumatera Utara , Sebagai: Wakil Gubernur . Tahun: 1999 - 2004</li><li>Pemerintah Provinsi Sumatera Utara , Sebagai: Sekretaris Daerah . Tahun: 1996 - 1999</li><li>Pemerintah Provinsi Sumatera Utara, Sebagai: Asisten Kesra . Tahun: 1992 - 1995</li><li>Pemer

In [33]:
c = profil_long.groupby(['akd_clean'])['noAnggota'].nunique().reset_index(name='cnt')
c['pct'] = round(100 * c['cnt'] / c.cnt.sum(),2)
c.sort_values('pct', ascending=[False])

,akd_clean,cnt,pct
2,Badan Anggaran,99,10.45
4,Badan Legislasi,76,8.03
19,Panitia Khusus,61,6.44
10,Komisi IV,55,5.81
12,Komisi V,53,5.60
3,Badan Kerja Sama Antar Parlemen,53,5.60
7,Komisi I,53,5.60
13,Komisi VI,53,5.60
16,Komisi X,52,5.49
15,Komisi VIII,52,5.49


## Pendekatan pada beberapa Kolom:
Ada beberapa kolom yang kita derive berdasarkan kolom yang ada di website DPR. Berikut kolom-kolom tersebut beserta tujuannya:
* is_dewan: pendekatan untuk mengetahui apakah seorang anggota DPR sebelumnya pernah menjabat menjadi anggota dewan, baik di DPR, MPR, DPD maupun di DPRD. Data ini derive dari kolom pekerjaan dari masing-masing anggota DPR.
* pendidikan_terakhir: pendekatan yg diambil berdasarkan biodata pendidikan yang ada di website DPR dari masing-masing anggota. Selanjutnya diambil pendidikan terakhir yang ada di kolom pendidikan.
* is_kader: diderive dari kolom organisasi. Tujuannya untuk mengetahui apakah anggota DPR dari suatu partai sebelumnya sudah pernah memegang jabatan dalam partai.
* usia_kategori: usia per tahun 2020 (saat script ini di run). Selanjutnya dibuat binning berdasarkan quintile (5 bin sama besar) agar sebaran usia tidak lagi numerical.

## Univariate Findings:
1. Partai tua nasionalis masih menjadi favorit. PDIP di urutan pertama diikuti golkar di urutan kedua. Partai islamis anjlok menempati posisi 3 terbawah.
2. 5 agama terbesar di Indonesia memiliki wakil di DPR, dengan jumlah proporsi yang hampir mirip (https://id.wikipedia.org/wiki/Agama_di_Indonesia)
3. Lebih dari 50% anggota DPR berpendidikan minimal S2.
4. Lebih dari 50% anggota DPR saat ini memiliki pengalaman menjabat sebagai anggota dewan sebelumnya.
5. Mayoritas anggota DPR tidak pernah memegang jabatan partai di partai yang mengusung mereka.
6. Rata-rata usia anggota DPR ialah 51.75 tahun. Anggota termuda berumur 24 tahun dari partai Nasdem dapil SULUT dan anggota tertua berumur 81 tahun dari partai Demokrat dapil SUMUT 1. 
7. Seorang anggota DPR dapat ditempatkan pada satu atau lebih Alat Kelengkapan Dewan (AKD). Badan Anggaran dan Badan Legislasi menjadi badan yang paling vital. Sebaliknya, komisi 2 menjadi komisi yang tidak difavoritkan. (http://www.dpr.go.id/akd/komisi)

In [34]:
def bivariate_analysis(col, sortcol = 'p(B|A)'):
    c = profil_long.groupby(['partai','rank_partai', col])['noAnggota'].nunique().reset_index(name='cnt')
    c['p(A|B)'] = c['cnt'] / c.groupby('partai')['cnt'].transform('sum')
    c['p(B|A)'] =  (c['cnt'] / c.groupby(col)['cnt'].transform('sum'))
    return c[c.cnt>0].sort_values(['rank_partai', sortcol], ascending=[True, False])

In [35]:
bivariate_analysis('akd_clean', 'cnt')

,partai,rank_partai,akd_clean,cnt,p(A|B),p(B|A)
98,PDIP,1.0,Badan Anggaran,21,0.108247,0.212121
100,PDIP,1.0,Badan Legislasi,15,0.077320,0.197368
99,PDIP,1.0,Badan Kerja Sama Antar Parlemen,12,0.061856,0.226415
105,PDIP,1.0,Komisi III,12,0.061856,0.230769
106,PDIP,1.0,Komisi IV,12,0.061856,0.218182
108,PDIP,1.0,Komisi V,12,0.061856,0.226415
109,PDIP,1.0,Komisi VI,12,0.061856,0.226415
110,PDIP,1.0,Komisi VII,12,0.061856,0.235294
111,PDIP,1.0,Komisi VIII,12,0.061856,0.230769
101,PDIP,1.0,Badan Musyawarah,11,0.056701,0.229167


In [36]:
bivariate_analysis('is_dewan')

,partai,rank_partai,is_dewan,cnt,p(A|B),p(B|A)
15,PDIP,1.0,dewan,93,0.732283,0.322917
16,PDIP,1.0,no_data,4,0.031496,0.250000
17,PDIP,1.0,non-dewan,30,0.236220,0.111940
3,Golkar,2.0,dewan,46,0.541176,0.159722
5,Golkar,2.0,non-dewan,37,0.435294,0.138060
4,Golkar,2.0,no_data,2,0.023529,0.125000
2,Gerindra,3.0,non-dewan,55,0.723684,0.205224
0,Gerindra,3.0,dewan,20,0.263158,0.069444
1,Gerindra,3.0,no_data,1,0.013158,0.062500
8,Nasdem,4.0,non-dewan,35,0.593220,0.130597


In [37]:
bivariate_analysis('agama')

,partai,rank_partai,agama,cnt,p(A|B),p(B|A)
17,PDIP,1.0,Hindu,7,0.056911,0.636364
20,PDIP,1.0,Kristen,32,0.260163,0.615385
16,PDIP,1.0,Budha,2,0.016260,0.500000
19,PDIP,1.0,Katolik,11,0.089431,0.458333
18,PDIP,1.0,Islam,71,0.577236,0.152688
3,Golkar,2.0,Budha,1,0.012048,0.250000
4,Golkar,2.0,Hindu,2,0.024096,0.181818
6,Golkar,2.0,Katolik,4,0.048193,0.166667
5,Golkar,2.0,Islam,72,0.867470,0.154839
7,Golkar,2.0,Kristen,4,0.048193,0.076923


In [38]:
profil_raw.groupby(['partai','rank_partai'])['usia'].agg(['mean', 'max', 'min'])


,,mean,max,min
partai,rank_partai,,,
Gerindra,3.0,51.373333,71.0,25.0
Golkar,2.0,52.168675,74.0,26.0
Nasdem,4.0,49.568966,72.0,24.0
PAN,8.0,49.738095,71.0,24.0
PD,6.0,51.764706,81.0,26.0
PDIP,1.0,53.040650,77.0,28.0
PKB,5.0,50.508772,75.0,30.0
PKS,7.0,54.666667,71.0,43.0
PPP,9.0,50.684211,65.0,29.0


In [39]:
bivariate_analysis('usia_kategori')

,partai,rank_partai,usia_kategori,cnt,p(A|B),p(B|A)
228,PDIP,1.0,"(55.0, 61.0]",27,0.219512,0.250000
227,PDIP,1.0,"(49.0, 55.0]",31,0.252033,0.244094
226,PDIP,1.0,"(44.0, 49.0]",24,0.195122,0.237624
229,PDIP,1.0,"(61.0, 81.0]",21,0.170732,0.221053
225,PDIP,1.0,"(23.0, 44.0]",20,0.162602,0.160000
53,Golkar,2.0,"(55.0, 61.0]",20,0.240964,0.185185
50,Golkar,2.0,"(23.0, 44.0]",21,0.253012,0.168000
54,Golkar,2.0,"(61.0, 81.0]",15,0.180723,0.157895
51,Golkar,2.0,"(44.0, 49.0]",13,0.156627,0.128713
52,Golkar,2.0,"(49.0, 55.0]",14,0.168675,0.110236


In [40]:
bivariate_analysis('pendidikan_terakhir')

,partai,rank_partai,pendidikan_terakhir,cnt,p(A|B),p(B|A)
29,PDIP,1.0,3_d3,3,0.023622,0.500000
30,PDIP,1.0,4_s1,51,0.401575,0.264249
32,PDIP,1.0,6_s3,22,0.173228,0.261905
27,PDIP,1.0,0_no_data,7,0.055118,0.189189
31,PDIP,1.0,5_s2,39,0.307087,0.178899
28,PDIP,1.0,2_sma,5,0.039370,0.156250
8,Golkar,2.0,5_s2,42,0.494118,0.192661
7,Golkar,2.0,4_s1,27,0.317647,0.139896
5,Golkar,2.0,0_no_data,5,0.058824,0.135135
9,Golkar,2.0,6_s3,8,0.094118,0.095238


In [41]:
c = bivariate_analysis('pendidikan_terakhir')
c[c.pendidikan_terakhir.str.contains('s3')].groupby('partai')['p(A|B)'].sum().reset_index().sort_values('p(A|B)', ascending=False)

,partai,p(A|B)
7,PKS,0.280000
8,PPP,0.263158
3,PAN,0.181818
5,PDIP,0.173228
4,PD,0.166667
0,Gerindra,0.144737
1,Golkar,0.094118
2,Nasdem,0.084746
6,PKB,0.034483


In [42]:
bivariate_analysis('is_kader', 'p(A|B)')

,partai,rank_partai,is_kader,cnt,p(A|B),p(B|A)
15,PDIP,1.0,eksternal,110,0.866142,0.341615
16,PDIP,1.0,internal,13,0.102362,0.055556
17,PDIP,1.0,no_data,4,0.031496,0.250000
4,Golkar,2.0,internal,51,0.600000,0.217949
3,Golkar,2.0,eksternal,32,0.376471,0.099379
5,Golkar,2.0,no_data,2,0.023529,0.125000
0,Gerindra,3.0,eksternal,46,0.605263,0.142857
1,Gerindra,3.0,internal,29,0.381579,0.123932
2,Gerindra,3.0,no_data,1,0.013158,0.062500
6,Nasdem,4.0,eksternal,33,0.559322,0.102484


## Bivariate Findings
P(A) adalah probabilitas dari kolom yang dipanggil dalam fungsi bivariate_analysis
P(B) adalah probabilitas dari partai

1. Sebaran AKD pada tiap partai hampir sama.Hanya saja, ada beberapa partai yang condong terhadap komisi-komisi tertentu terlihat dari jumlah penempatan anggotanya. Badan Anggaran dan Badan Legislasi menjadi vital dari semua partai. Terdapat pencilan pada partai PKB dimana mayoritas anggotanya masuk dalam panitia khusus. Pimpinan diambil 1 dari masing-masing 5 partai teratas. 
2. PDIP dan PPP adalah partai veteran dimana mayoritas anggotanya melanjutkan tugasnya sebagai anggota dewan. Bisa jadi ini menandakan PDIP sangat baik dalam menjaga suaranya di tingkat masyarakat sedangkan PPP memiliki kantung-kantung suara yang loyal. Dilain hal, Gerindra adalah partai pembaruan. Hal ini menjadi wajar mengingat Prabowo menjadi ikon pembaruan di pemilu kemarin. Mayoritas (72%) anggota Gerindra belum memiliki pengalaman menjadi anggota legislatif sebelumnya.
3. Hanya PDIP dan Golkar yang mewakili 5 agama mayoritas di Indonesia. Hal ini selaras dengan ideologi nasionalis yang mereka usung. Seorang caleg yang beragama non-islam lebih besar kemungkinannya terpilih via PDIP (metric P(B|A) terbesar). PKS dan PPP menjadi partai yang anggotanya 100% muslim.
4. PKS memiliki rerata usia anggota tertua dengan 54,67 tahun sedangkan Nasdem menjadi partai termuda dengan rerata usia 49,56. PDIP dan Golkar memiliki sebaran umur_kategorical yang cukup rata dimana selisih max dan min P(A|B) nya serupa, 0.09.
5. PKS mengokohkan dirinya menjadi partai terdidik dengan memiliki porsi S3 terbesar dalam partainya. PKB untuk S2, PDIP untuk S1, dan PPP untuk SMA. SMP adalah data outlier mengingat syarat minimum untuk menjadi anggota DPR ialah SMA.
6. Golkar, PKB, PKS dan PPP adalah partai kaderisasi. Minimal 60% anggotanya pernah menjabat di internal partai.

In [43]:
## given my profile, which party has the best chance for me to be picked as one of their legislative :)
c = profil_raw.groupby(['partai', 'rank_partai', 'agama', 'is_dewan', 'is_kader', 'pendidikan_terakhir'])['noAnggota'].nunique().reset_index(name='cnt')
c['p(A|B)'] = c['cnt'] / c.groupby('partai')['cnt'].transform('sum')
c['p(B|A)'] =  (c['cnt'] / c.groupby(['agama','is_dewan', 'is_kader', 'pendidikan_terakhir'])['cnt'].transform('sum'))
d = c.sort_values(['agama', 'is_dewan', 'is_kader', 'pendidikan_terakhir', 'p(B|A)'], ascending=[True, False, False, True,False])
d[(d.agama=='Islam') & (d.is_dewan=='non-dewan') & (d.is_kader=='eksternal') & (d.pendidikan_terakhir=='5_s2')]

,partai,rank_partai,agama,is_dewan,is_kader,pendidikan_terakhir,cnt,p(A|B),p(B|A)
8,Gerindra,3.0,Islam,non-dewan,eksternal,5_s2,10,0.133333,0.20
32,Golkar,2.0,Islam,non-dewan,eksternal,5_s2,10,0.120482,0.20
74,PAN,8.0,Islam,non-dewan,eksternal,5_s2,7,0.166667,0.14
142,PKB,5.0,Islam,non-dewan,eksternal,5_s2,7,0.122807,0.14
117,PDIP,1.0,Islam,non-dewan,eksternal,5_s2,6,0.048780,0.12
50,Nasdem,4.0,Islam,non-dewan,eksternal,5_s2,5,0.086207,0.10
161,PKS,7.0,Islam,non-dewan,eksternal,5_s2,3,0.062500,0.06
92,PD,6.0,Islam,non-dewan,eksternal,5_s2,2,0.039216,0.04


## Trivia
Partai mana yang paling besar memungkinkan saya (islam, non-dewan, eksternal, S2) lolos menjadi anggota DPR: Gerindra atau Golkar.
